In [25]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing import image_dataset_from_directory
from keras.models import Sequential
from keras.layers import Conv2D, AveragePooling2D, Flatten, Dense, Input, MaxPooling2D,  Dropout, Rescaling


def dataset_split(data, train_split=0.85, val_split=0.15, shuffle=True, shuffle_size=5700):
    if shuffle:
        data = data.shuffle(shuffle_size)
    data_size = len(data)
    train_size = int(data_size * train_split)
    train_set = data.take(train_size)
    val_size = int(data_size * val_split)
    remaining_data = data.skip(train_size)
    val_set = remaining_data.take(val_size)
    
    return train_set, val_set


training_data = image_dataset_from_directory( "Brain_tumor/Training", pad_to_aspect_ratio=True )
testing_data = image_dataset_from_directory( "Brain_tumor/Testing", pad_to_aspect_ratio=True )
train_set, val_set = dataset_split( data= training_data )

Found 5712 files belonging to 4 classes.
Found 1311 files belonging to 4 classes.


In [26]:
num_classes = 4

model = Sequential()

# Input layer
model.add(Input(shape=(256, 256, 3)))

# First Convolutional layer with ReLU
model.add(Conv2D(filters=6, kernel_size=(5, 5), padding='same', activation='relu'))

# Average Pooling layer
model.add(AveragePooling2D(pool_size=(2, 2), strides=2))

# Second Convolutional layer with ReLU
model.add(Conv2D(filters=16, kernel_size=(5, 5), activation='relu'))

# Average Pooling layer
model.add(AveragePooling2D(pool_size=(2, 2), strides=2))

# Flattening layer
model.add(Flatten())

# First Dense layer with ReLU
model.add(Dense(units=120, activation='relu'))  # Updated to 100 neurons

# Second Dense layer with ReLU
model.add(Dense(units=84, activation='relu'))  # Updated to 75 neurons

# Output layer with Softmax
model.add(Dense(units=10, activation='softmax'))  # Output layer neurons set to number of classes

# Compile the model
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Model summary to verify the architecture
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 256, 256, 6)    │           456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_10            │ (None, 128, 128, 6)    │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 124, 124, 16)   │         2,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_11            │ (None, 62, 62, 16)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 61504)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 120)            │     7,380,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 84)             │        10,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 10)             │           850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,394,486 (28.21 MB)

 Trainable params: 7,394,486 (28.21 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
history1 = model.fit( train_set, validation_data=val_set, epochs=20 )

Epoch 1/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 26s 150ms/step - accuracy: 0.2427 - loss: nan - val_accuracy: 0.2272 - val_loss: nan
Epoch 2/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 24s 141ms/step - accuracy: 0.2336 - loss: nan - val_accuracy: 0.2392 - val_loss: nan
Epoch 3/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 22s 129ms/step - accuracy: 0.2341 - loss: nan - val_accuracy: 0.2308 - val_loss: nan
Epoch 4/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 22s 128ms/step - accuracy: 0.2268 - loss: nan - val_accuracy: 0.1971 - val_loss: nan
Epoch 5/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 22s 127ms/step - accuracy: 0.2247 - loss: nan - val_accuracy: 0.2440 - val_loss: nan
Epoch 6/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 21s 126ms/step - accuracy: 0.2300 - loss: nan - val_accuracy: 0.2236 - val_loss: nan
Epoch 7/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 21s 126ms/step - accuracy: 0.2338 - loss: nan - val_accuracy: 0.2452 - val_loss: nan
Epoch 8/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 21s 125ms/step - accuracy: 0.2276 - loss: nan - val_accuracy: 0.2356 - val_loss: nan


In [34]:
training_accuracy = history1.history['accuracy']
validation_accuracy = history1.history['val_accuracy']
training_accuracy = [ round( num, 3 ) for num in training_accuracy ]
validation_accuracy = [ round( num, 3 ) for num in validation_accuracy ]

print( training_accuracy )
print( validation_accuracy )

[0.228, 0.229, 0.233, 0.233, 0.231, 0.229, 0.23, 0.23, 0.233, 0.23, 0.232, 0.23, 0.232, 0.235, 0.229, 0.229, 0.231, 0.229, 0.231, 0.231]
[0.227, 0.239, 0.231, 0.197, 0.244, 0.224, 0.245, 0.236, 0.237, 0.24, 0.214, 0.272, 0.212, 0.227, 0.222, 0.222, 0.218, 0.218, 0.24, 0.219]


In [35]:
num_classes = 4

model2 = Sequential()

# Insert Rescaling layer
model2.add(Rescaling(1./255, input_shape=(256, 256, 3)))

# First Convolutional layer with ReLU
model2.add(Conv2D(filters=6, kernel_size=(5, 5), padding='same', activation='relu'))

# Replace with MaxPooling
model2.add(MaxPooling2D(pool_size=(2, 2), strides=2))

# Second Convolutional layer with ReLU
model2.add(Conv2D(filters=16, kernel_size=(5, 5), activation='relu'))

# Replace with MaxPooling
model2.add(MaxPooling2D(pool_size=(2, 2), strides=2))

# Flatten before passing to the Dense layers
model2.add(Flatten())

# First Dense layer with ReLU
model2.add(Dense(units=120, activation='relu'))

# Insert Dropout layer here with a 0.2 dropout rate
model2.add(Dropout(0.2))

# Second Dense layer with ReLU
model2.add(Dense(units=84, activation='relu'))

# Insert Dropout layer here with a 0.2 dropout rate
model2.add(Dropout(0.2))

# Output layer with Softmax
# Make sure the number of units equals the number of classes
model2.add(Dense(units=10, activation='softmax'))

# Compile the model
model2.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Model summary to verify the architecture
model2.summary()


C:\Users\zafee\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:18: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_1 (Rescaling)         │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 256, 256, 6)    │           456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 128, 128, 6)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 124, 124, 16)   │         2,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 62, 62, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 61504)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 120)            │     7,380,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 120)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 84)             │        10,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 84)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 10)             │           850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,394,486 (28.21 MB)

 Trainable params: 7,394,486 (28.21 MB)

 Non-trainable params: 0 (0.00 B)

In [36]:
history2 = model2.fit( train_set, validation_data=val_set, epochs=20 )

Epoch 1/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 23s 132ms/step - accuracy: 0.3911 - loss: 1.5133 - val_accuracy: 0.6587 - val_loss: 0.8267
Epoch 2/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 23s 131ms/step - accuracy: 0.5938 - loss: 0.9826 - val_accuracy: 0.7452 - val_loss: 0.6413
Epoch 3/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 23s 131ms/step - accuracy: 0.7161 - loss: 0.7401 - val_accuracy: 0.7990 - val_loss: 0.5178
Epoch 4/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 23s 135ms/step - accuracy: 0.7555 - loss: 0.6314 - val_accuracy: 0.8618 - val_loss: 0.3909
Epoch 5/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 22s 130ms/step - accuracy: 0.8079 - loss: 0.5255 - val_accuracy: 0.8401 - val_loss: 0.4042
Epoch 6/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 22s 131ms/step - accuracy: 0.8276 - loss: 0.4762 - val_accuracy: 0.8377 - val_loss: 0.4152
Epoch 7/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 22s 130ms/step - accuracy: 0.8535 - loss: 0.4007 - val_accuracy: 0.8654 - val_loss: 0.3343
Epoch 8/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 22s 131ms/step - accuracy: 0.8589 - loss: 0

In [37]:
training_accuracy2 = history2.history['accuracy']
validation_accuracy2 = history2.history['val_accuracy']
training_accuracy2 = [ round( num, 3 ) for num in training_accuracy2 ]
validation_accuracy2 = [ round( num, 3 ) for num in validation_accuracy2 ]

print( training_accuracy2 )
print( validation_accuracy2 )

[0.476, 0.635, 0.726, 0.772, 0.812, 0.837, 0.854, 0.866, 0.883, 0.898, 0.91, 0.916, 0.933, 0.94, 0.95, 0.953, 0.956, 0.967, 0.965, 0.971]
[0.659, 0.745, 0.799, 0.862, 0.84, 0.838, 0.865, 0.925, 0.892, 0.941, 0.962, 0.974, 0.921, 0.98, 0.962, 0.987, 0.99, 0.99, 0.994, 0.969]
